In [13]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [14]:
df_trades = pd.read_pickle("USD_JPY_H4_trades.pkl")

In [15]:
df_trades.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2018-01-02T02:00:00.000000000Z,1009,112.720,112.751,112.666,112.688,112.712,112.744,112.657,112.683,...,112.582,112.800,112.559,0.211,-1,1,1,112.8211,112.7367,112.9899
12,2018-01-04T02:00:00.000000000Z,1560,112.731,112.731,112.618,112.661,112.724,112.724,112.611,112.654,...,112.480,112.785,112.455,0.298,-1,1,1,112.8148,112.6956,113.0532
17,2018-01-04T22:00:00.000000000Z,1563,112.750,112.806,112.730,112.774,112.722,112.799,112.708,112.767,...,112.713,112.846,112.706,0.125,1,-1,-1,112.6935,112.7435,112.5935
21,2018-01-05T14:00:00.000000000Z,3844,113.213,113.276,113.082,113.160,113.206,113.269,113.076,113.153,...,113.021,113.330,112.999,0.291,-1,-1,-1,112.9699,113.0863,112.7371
27,2018-01-08T14:00:00.000000000Z,2537,112.928,113.125,112.904,113.082,112.921,113.119,112.898,113.076,...,112.884,113.173,112.877,0.283,1,-1,-1,112.8487,112.9619,112.6223


In [16]:
pair = "USD_JPY"

In [17]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [18]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [19]:
df_trades["time"] = [parse(x) for x in df_trades.time] 
df_raw["time"] = [parse(x) for x in df_raw.time] 

In [20]:
df_trades["next"] = df_trades["time"].shift(-1)

In [21]:
df_trades['trade_end'] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [22]:
df_trades[['time', 'next', 'trade_end', 'trade_start']].head()

,time,next,trade_end,trade_start
0,2018-01-02 02:00:00+00:00,2018-01-04 02:00:00+00:00,2018-01-04 05:55:00+00:00,2018-01-02 06:00:00+00:00
12,2018-01-04 02:00:00+00:00,2018-01-04 22:00:00+00:00,2018-01-05 01:55:00+00:00,2018-01-04 06:00:00+00:00
17,2018-01-04 22:00:00+00:00,2018-01-05 14:00:00+00:00,2018-01-05 17:55:00+00:00,2018-01-05 02:00:00+00:00
21,2018-01-05 14:00:00+00:00,2018-01-08 14:00:00+00:00,2018-01-08 17:55:00+00:00,2018-01-05 18:00:00+00:00
27,2018-01-08 14:00:00+00:00,2018-01-09 18:00:00+00:00,2018-01-09 21:55:00+00:00,2018-01-08 18:00:00+00:00


In [23]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [24]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def end_hit_calc(direction, SL, price, start_price):
    delta = price - start_price
    full_delta = start_price - SL
    fraction = abs(delta / full_delta)
   
    if direction == 1 and price >= start_price:
        return fraction
    elif direction == 1 and price < start_price:
        return -fraction
    elif direction == -1 and price <= start_price:
        return fraction
    elif direction == -1 and price > start_price:
        return -fraction


def process_buy(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(ask_prices):
        if triggered(1, price, entry_price) == True:
            for live_price in bid_prices[index:]:
                if live_price >= TP:
                    return 2.0
                elif live_price <= SL:
                    return -1.0
            return end_hit_calc(1, SL, live_price, entry_price)
    return 0.0

def process_sell(TP, SL, ask_prices, bid_prices, entry_price):
    for index, price in enumerate(bid_prices):
        if triggered(-1, price, entry_price) == True:
            for live_price in ask_prices[index:]:
                if live_price <= TP:
                    return 2.0
                elif live_price >= SL:
                    return -1.0
            return end_hit_calc(-1, SL, live_price, entry_price)
    return 0.0


In [27]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    if row.SIGNAL == 1:
        r = process_buy(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
    else:
        r = process_sell(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
print(total)

43.36120153064007


In [26]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.trade_start) & (df_raw.time <= row.trade_end)]
    if row.SIGNAL == 1:
        r = process_buy(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
    else:
        r = process_sell(row.TAKEPROFIT, row.STOPLOSS, m5_data.ask_c.values, m5_data.bid_c.values, row.ENTRY)
        total += r
print(total)

43.36120153064007
